# MNIST check

In [1]:
using Pkg
using DelimitedFiles
using BenchmarkTools
using PyPlot

In [2]:
Pkg.activate("../Sampling/")
using Sampling
Pkg.activate("../../GaussianEP/")
using GaussianEP

  Activating environment at `~/GaussianEP/Sampling/Project.toml`
┌ Info: Precompiling Sampling [89856a0a-5ec6-497b-a605-6c7f2f763f9e]
└ @ Base loading.jl:1342
  Activating environment at `~/GaussianEP/Project.toml`


## (Pv,Ph) = (Bernoulli, Bernoulli)

In [3]:
using JLD2

mean_data = readdlm("../MNIST/mean.txt", '\t', Float64, '\n')
covv_data = readdlm("../MNIST/corr.txt", '\t', Float64, '\n')
data_EP = load("../MNIST/best_w_sofar.jld2")
#data_EP = load("../MNIST/res_BerRBM_rh8_w0bim_adagrad_SGD_epTRBL.jld2");
#w_EP = data_EP["res"].weights;

w_EP = data_EP["res_prior"].res.weights;
P0 = data_EP["res_prior"].prior;

N,M = size(w_EP)

┌ Warning: saved type EPState{Float64} is missing field av_cav in workspace type; reconstructing
└ @ JLD2 /home/disat/amuntoni/.julia/packages/JLD2/DcnTD/src/data/reconstructing_datatypes.jl:152
┌ Warning: type parameters for NamedTuple{(:weights, :likelihood, :state, :opt, :grad),Tuple} do not match type NamedTuple in workspace; reconstructing
└ @ JLD2 /home/disat/amuntoni/.julia/packages/JLD2/DcnTD/src/data/reconstructing_datatypes.jl:430
┌ Warning: type epRBM.Adam{Float64,Int64} does not exist in workspace; reconstructing
└ @ JLD2 /home/disat/amuntoni/.julia/packages/JLD2/DcnTD/src/data/reconstructing_datatypes.jl:450


(784, 400)

In [ ]:

N,M = size(w_EP)

#rhov = 0.8677
#rhoh = 0.8
#Pv = BinaryPrior(0.0, 1.0, rhov)
#Ph = BinaryPrior(0.0, 1.0, rhoh)
y = zeros(N+M)

#P0 = vcat([Pv for i = 1:N], [Ph for i = 1:M]);
H = GaussianEP.TermRBM(zeros(N,M),y,1.0);

#out_ep = expectation_propagation(H,P0, nprint = 1000, maxiter = 10000, damp = 0.99, epsconv = 1e-5);

beta_min = 0.01
beta_max = 1.0
niter = 50
step = (beta_max - beta_min) / niter
beta = beta_min
out_ep = nothing



for iter = 1:niter+1
    H.w .= beta * w_EP
    println(beta)
    if iter == 1
        out_ep = expectation_propagation([H],P0,nprint = 100, damp= 0.999, epsconv = 1e-4, maxiter = 10000, state = nothing);
    else
        out_ep = expectation_propagation([H],P0,nprint = 100, damp= 0.999, epsconv = 1e-4, maxiter = 10000, state = out_ep[1].state);
    end
    beta += step
end 


In [ ]:

N,M = size(w_EP)
y = zeros(N+M)
H = GaussianEP.TermRBM(w_EP,y,1.0);
out_ep0 = expectation_propagation([H],P0,nprint = 100, damp= 0.999, epsconv = 1e-4, maxiter = 10000, state = nothing)

In [4]:
y = zeros(N+M)
H = GaussianEP.TermRBM(w_EP,y,1.0);
vstate, hstate, S_vh, _, _ = GaussianEP.block_expectation_propagation([H], P0, nprint = 1000, damp= 0.99, epsconv = 5e-5, maxiter = 10000);

In [ ]:
[P0[end].ρ, P0[1].x0, P0[1].x1]

P0v = [Sampling.BinaryPrior(0.0, 1.0, P0[i].ρ) for i in 1:N];
P0h = [Sampling.BinaryPrior(0.0, 1.0, P0[i+N].ρ) for i in 1:M];


In [ ]:
Ns = 10000
Neq = 100000
x0 = zeros(N+M)
x = zeros(Ns,N+M)

Sampling.gibbssampling!(x0, w_EP, P0v, P0h, Neq)
Nwait = 100
x[1,:] .= x0
for i = 2:Ns
    x[i,:] .= Sampling.gibbssampling(x[i-1,1:N], x[i-1,N+1:end], w_EP, P0v, P0h, Nwait)
end

In [ ]:
using Statistics
mean_mc = mean(x,dims = 1)
cov_mc = cov(x)
vcov_mc = cov_mc[1:N,1:N];

In [ ]:
scatter(mean_data, vstate.av)

In [ ]:
scatter(mean_data, out_ep0[1].av[1:N])

In [ ]:
scatter(mean_data, mean_mc[1:N])

In [ ]:
scatter(vstate.av, mean_mc[1:N])


In [ ]:
p2 = scatter(vec(vstate.Σ), vec(vcov_mc))

In [ ]:
[vstate.av out_ep0[1].av[1:N]]
using Plots

p1 = histogram(out_ep0[1].av[1:N], yscale = :log10, label ="ep")
p2 = histogram(vstate.av[1:N], yscale = :log10, label = "block")
p3 = histogram(out_ep[1].av[1:N], yscale = :log10, label = "ann")
Plots.plot(p1,p2, p3)

In [ ]:

p1 = Plots.plot(mean_data,vstate.av, seriestype = :scatter, label = "EPblock", xlabel = "MNIST av")
p2 = Plots.plot(mean_data, out_ep0[1].av[1:N], seriestype = :scatter, label = "EP", xlabel = "MNIST av")
p3 = Plots.plot(mean_data, out_ep[1].av[1:N], seriestype = :scatter, label = "EP anneal", xlabel = "MNIST av")
Plots.plot(p1,p2,p3)



In [ ]:
vcov_data = vec(covv_data)
vcov_ep = vec(vstate.Σ)
Plots.plot(vcov_data, vcov_ep, seriestype = :scatter)

In [ ]:

using BoltzmannMachines
using JLD2

data_bm = load("../MNIST/bm_rbm_cdstep10_batchsize500.jld2")
w_BM = data_bm["rbm"].weights;

In [ ]:
using LinearAlgebra
e = eigen(w_BM'*w_BM)
e.values

In [ ]:
using BoltzmannMachines
#data_bm = load("../MNIST/winf_fields_BerRBM.jld2")
data_bm = load("../MNIST/bm_rbm_cdstep10_batchsize500.jld2")
w_BM = data_bm["rbm"].weights
N,M = size(w_BM)
y = zeros(N+M,)
rhov_BM = 1.0 ./ (1.0 .+ exp.(data_bm["rbm"].visbias));
rhoh_BM = 1.0 ./ (1.0 .+ exp.(data_bm["rbm"].hidbias));

Pv = [BinaryPrior(0.0,1.0,rhov_BM[i]) for i = 1:N]
Ph = [BinaryPrior(0.0,1.0,rhoh_BM[i]) for i = 1:M]

P0_BM = vcat(Pv,Ph);

H_BM = GaussianEP.TermRBM(zeros(N,M),y,1.0);
beta_min = 0.05
beta_max = 1.0
niter = 500
step = (beta_max - beta_min) / niter
beta = beta_min
out_BM = nothing

#out_BM_plain = expectation_propagation([H_BM],P0_BM,nprint = 100, damp= 0.995, epsconv = 5e-4, maxiter = 3000, state = nothing);

for iter = 1:niter+1
    H_BM.w .= beta * w_BM
    println(beta)
    if iter == 1
        out_BM = expectation_propagation([H_BM],P0_BM,nprint = 1000, damp= 0.995, epsconv = 2e-3, maxiter = 10000, state = nothing);
    else
        out_BM = expectation_propagation([H_BM],P0_BM,nprint = 1000, damp= 0.995, epsconv = 2e-3, maxiter = 10000, state = out_BM[1].state);
    end
    beta += step
end 



In [ ]:
#data_bm = load("../MNIST/bm_rbm_cdstep10_batchsize500.jld2")
#w_BM = data_bm["rbm"].weights
p1 = hist(vec(w_BM), bins = 200, density = true)
p2 = hist(vec(w_EP), bins = 200, density = true)
#p2 = plot(vec(H_BM.w),vec(w_BM), seriestype= :scatter)
#l = @layout [a;b]
plot(p1,p2)

In [ ]:
using Random


N,M = size(w_BM)
rhov = 0.8677
rhoh = 0.8
biasv_ep = log((1.0 - rhov)/rhov)
biash_ep = log((1.0 - rhoh)/rhoh)
rbm_bm = data_bm["rbm"]
rbm_ep = BernoulliRBM(w_EP, fill(biasv_ep, N), fill(biash_ep, M))
particle_ep = BoltzmannMachines.initparticles(rbm_ep, 1, biased=true)
particle_bm = BoltzmannMachines.initparticles(rbm_bm, 1, biased=true)

#typeof(rbm_bm)
Nconf = 5000
Neq = 500000
samples_ep = zeros(Nconf, N)
samples_bm = zeros(Nconf, N)

hidden_bm = zeros(Nconf,M)
hidden_ep = zeros(Nconf,M)

myseed = 19
Random.seed!(myseed)
gibbssample!(particle_ep, rbm_ep, Neq)
gibbssample!(particle_bm, rbm_bm, Neq)
for i = 1:Nconf

    gibbssample!(particle_ep, rbm_ep, 1)
    gibbssample!(particle_bm, rbm_bm, 1)
    samples_ep[i,:] = particle_ep[1]
    samples_bm[i,:] = particle_bm[1]
    hidden_ep[i,:] = particle_ep[2]
    hidden_bm[i,:] = particle_bm[2]
end


In [ ]:

Pv_mc = [Sampling.BinaryPrior(0.0,1.0,rhov_BM[i]) for i = 1:N]
Ph_mc = [Sampling.BinaryPrior(0.0,1.0,rhoh_BM[i]) for i = 1:M]

P0mc= vcat(Pv_mc,Ph_mc);

mc_out = Sampling.MC_sim(w_BM, P0mc, Neq, 0.0, N_iter = 5000)
av_mc, va_mc, cov_mc, covv_mc = Sampling.compute_statistics(mc_out, N, M);

In [ ]:
mean_bm_mc = mean(samples_bm, dims=1)
p1 = plot(mean_data, out_BM[1].av[1:N], seriestype = :scatter, xlabel = "Data", ylabel = "EP (BM par)")
p2 = plot(mean_data, vec(mean_bm_mc), seriestype = :scatter, xlabel = "Data", ylabel = "MC from BMs.jl (BM par)")
p3 = plot(mean_data, av_mc, seriestype = :scatter, xlabel = "Data", aspect_ratio = :equal, ylabel = "MC from Sampling.jl (BM par)")
l = @layout  [a{0.4h}  b{0.4h} c{0.4h}]
plot(p1,  p2, p3, layout = l)

In [ ]:
Ndig = 100
digits_ep = zeros(Ndig, N)
digits_bm = zeros(Ndig, N)
L = convert(Int64,sqrt(N))

p1 = []
p2 = []
idx = 90
for i = 1:Ndig
    digits_bm[i,:] = samplevisible(rbm_bm, hidden_bm[idx,:])
    digits_ep[i,:] = samplevisible(rbm_ep, hidden_ep[idx,:])
    p1 = vcat(p1, matshow(reshape(mean(digits_ep, dims=1)', L, L), aspect_ratio = :equal, yflip = false))
    p2 = vcat(p2, matshow(reshape(mean(digits_bm, dims=1)', L, L), aspect_ratio = :equal, yflip = false))

end



In [ ]:
idx = 100
plot(p1[idx], p2[idx])

In [ ]:

Pv_mc = [Sampling.BinaryPrior(0.0,1.0,rhov) for i = 1:N]
Ph_mc = [Sampling.BinaryPrior(0.0,1.0,rhoh) for i = 1:M]

P0mc= vcat(Pv_mc,Ph_mc);

mc_out = Sampling.MC_sim(w_EP, P0mc, Neq, 0.0, N_iter = 10000)
av_mc, va_mc, cov_mc, covv_mc = Sampling.compute_statistics(mc_out, N, M);

In [ ]:
mean_ep_mc = mean(samples_ep, dims=1)
p1 = plot(mean_data, out_ep[1].av[1:N], seriestype = :scatter, xlabel = "Data", ylabel = "EP (EP par)")
p2 = plot(mean_data, vec(mean_ep_mc), seriestype = :scatter, xlabel = "Data", ylabel = "MC from BMs.jl (EP par)")
p3 = plot(mean_data, av_mc, seriestype = :scatter, xlabel = "Data", aspect_ratio = :equal, ylabel = "MC from Sampling.jl (EP par)")
l = @layout  [a{0.4h}  b{0.4h} c{0.4h}]
plot(p1,  p2, p3, layout = l)

# Trash hereafter:

In [ ]:
covv_ep = zeros(N,N)
for i = 1:N, j = i+1:N
	covv_ep[i,j] = out_ep.state.Σ[i,j] + out_ep.state.av[i] * out_ep.state.av[j]
	covv_ep[j,i] = covv_ep[i,j]
end

va_data = zeros(N,)
for i = 1:N
	va_data[i] = covv_data[i,i] - mean_data[i] * mean_data[i]
	covv_data[i,i] = NaN
end

covv_bm = zeros(N,N)
for i = 1:N, j = i+1:N
	covv_bm[i,j] = out_BM.state.Σ[i,j] + out_BM.state.av[i] * out_BM.state.av[j]
	covv_bm[j,i] = covv_bm[i,j]
end


In [ ]:
p = Plots.plot(mean_data, mean_data, aspect_ratio = :equal, seriestype = :scatter, label = "x = y", legend = :topleft, xlabel= "Data av")
p = Plots.plot!(mean_data, out_ep.av[1:N], aspect_ratio = :equal, seriestype = :scatter, label = "EP means", legend = :topleft, xlabel= "Data av")
p1 = Plots.plot(mean_data, mean_data, aspect_ratio = :equal, seriestype = :scatter, label = "x = y", legend = :topleft, xlabel= "Data av")
p1 = Plots.plot!(mean_data, out_BM.av[1:N], seriestype = :scatter, label = "BM means", legend = :topleft, xlabel= "Data av")
p2 = Plots.plot(out_BM.av[1:N], out_ep.av[1:N], seriestype = :scatter, label = "Av", legend = :topleft, xlabel = "BM", ylabel = "EP")
l = @layout [a{0.3h} b{0.3h} c{0.3h}]
Plots.plot(p, p1,p2, layout = l)

In [ ]:
p = Plots.plot(va_data, va_data, aspect_ratio = :equal, seriestype = :scatter, label = "x = y", legend = :topleft, xlabel= "Data var")
p = Plots.plot!(va_data, out_ep.va[1:N], aspect_ratio = :equal, seriestype = :scatter, label = "EP vars", legend = :topleft, xlabel= "Data var")
p1 = Plots.plot(va_data, va_data, aspect_ratio = :equal, seriestype = :scatter, label = "x = y", legend = :topleft, xlabel= "Data var")
p1 = Plots.plot!(va_data, out_BM.va[1:N], seriestype = :scatter, label = "BM vars", legend = :topleft, xlabel= "Data var")
p2 = Plots.plot(out_BM.va[1:N], out_ep.va[1:N], seriestype = :scatter, label = "Vars", legend = :topleft, xlabel = "BM", ylabel = "EP")
l = @layout [a{0.3h} b{0.3h} c{0.3h}]
Plots.plot(p, p1,p2, layout = l)

In [ ]:
vcovv_data = vec(covv_data)
vcovv_ep = vec(covv_ep)
vcovv_bm = vec(covv_bm)

p = Plots.plot(vcovv_data, vcovv_data, aspect_ratio = :equal, seriestype = :scatter, label = "x = y", legend = :topleft, xlabel= "Data cov(v,v)")
p = Plots.plot!(vcovv_data, vcovv_ep, aspect_ratio = :equal, seriestype = :scatter, label = "EP covv", legend = :topleft, xlabel= "Data cov(v,v)")
p1 = Plots.plot(vcovv_data, vcovv_data, aspect_ratio = :equal, seriestype = :scatter, label = "x = y", legend = :topleft, xlabel= "Data cov(v,v)")
p1 = Plots.plot!(vcovv_data, vcovv_bm, seriestype = :scatter, label = "BM covv", legend = :topleft, xlabel= "Data cov(v,v)")
p2 = Plots.plot(vcovv_bm, vcovv_ep, seriestype = :scatter, label = "Cov (v,v)", legend = :topleft, xlabel = "BM", ylabel = "EP")
l = @layout [a{0.3h} b{0.3h} c{0.3h}]
Plots.plot(p, p1,p2, layout = l)



In [ ]:
writedlm(open("../MNIST/covv_bm.dat", "w"), vec(covv_bm))
writedlm(open("../MNIST/covv_ep.dat", "w"), vec(covv_ep))
writedlm(open("../MNIST/covv_data.dat", "w"), vec(covv_data))
size(vec(covv_bm))